In [1]:
import pandas as pd
import numpy as np

import urllib2, urllib
import json
import pandas as pd
from cpyMSpec import IsotopePattern

def _combine_results(dfs):  
    if len(dfs) == 0:
        return []
    
    df = pd.concat(dfs)
    df['abs_ppm'] = abs(df['ppm'])
    df = df.sort_values(by='abs_ppm')
    
    df = df[df['unsat'] >= 0]
    
    del df['abs_ppm']
    df.index = range(len(df))

    return df[['em', 'error', 'mf', 'ppm', 'unsat', 'adduct']]

def search_mz_candidates_chemcalc(mass, adducts, ppm_limit=5, charge=1):
    chemcalcURL = 'http://www.chemcalc.org/chemcalc/em'
    mfRange = ''.join(['C0-100','H0-100','N0-10','O0-10', 'S0-5', 'P0-5'])
    dfs = []
    for adduct in adducts:
        mass_ = mass - IsotopePattern(adduct).charged(charge).masses[0]
        params = {'mfRange': mfRange,'monoisotopicMass': mass_, 'massRange': ppm_limit*mass*1e-6}
        response = urllib2.urlopen(chemcalcURL, urllib.urlencode(params))

        data = json.loads(response.read())
        if data['results'] == []:
            continue
        df = pd.DataFrame.from_dict(data['results'])
        df['abs_ppm'] = abs(df['ppm'])
        df['adduct'] = adduct
        del df['info']
        dfs.append(df)

    return _combine_results(dfs)

While ChemCalc is convenient to use, it requires internet access and relies on availability of an external service.

I tried a local alternative to it, R package 'Rdisop', which can be accessed from Python through rpy2.

In [4]:
def search_mz_candidates_rdisop(mass, adducts, ppm_limit=5, charge=1):
    import rpy2.robjects.packages as rpackages
    rdisop = rpackages.importr('Rdisop')

    from rpy2.robjects import numpy2ri
    numpy2ri.activate()

    maxEl = "C100H100N10O10S5P5"
    
    dfs = []
    for adduct in adducts:
        mass_ = mass - IsotopePattern(adduct).charged(charge).masses[0]
        allowed = rdisop.initializeElements(['C', 'H', 'N', 'O', 'P', 'S'])
        result = rdisop.decomposeMass(mass_, ppm=ppm_limit, z=0,
                                      maxElements=maxEl, elements=allowed)
        #print result
        df = pd.DataFrame.from_dict({'mf': np.array(result[0]),
                                    'em': np.array(result[2]),
                                    'score': np.array(result[1]),
                                    'valid': np.array(result[5]) == "Valid",
                                    'unsat': np.array(result[6], dtype=int)})
   
        df['error'] = df['em'] - mass_
        df['ppm'] = df['error'] / mass_ * 1e6

        df['adduct'] = adduct
        df = df[df['valid']]
        del df['valid']

        dfs.append(df)
        
    return _combine_results(dfs)
  

In [5]:
m = 835.135
%time search_mz_candidates_chemcalc(m, ['H', 'K', 'Na'], ppm_limit=1)[:10]

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 67.6 ms


,em,error,mf,ppm,unsat,adduct
0,812.145770,-0.000009,C52H16N10O2,-0.0112,50,Na
1,796.171832,-0.000010,C46H37O7PS2,-0.0120,29,K
2,796.171866,0.000025,C31H41N8O7PS4,0.0310,16,K
3,834.127750,0.000027,C47H19N10O5P,0.0322,44,H
4,812.145740,-0.000039,C25H57N2O7P5S5,-0.0475,1,Na
5,796.171802,-0.000040,C30H47N4O9P3S3,-0.0498,11,K
6,796.171890,0.000048,C32H36N10O7P4,0.0608,22,K
7,812.145835,0.000056,C42H41N2O3PS5,0.0688,24,Na
8,834.127662,-0.000061,C45H30N4O7S3,-0.0734,33,H
9,834.127656,-0.000068,C30H35N10O9P5,-0.0811,21,H


Calling Rdisop turned out to be slower than asking Chemcalc, at least for largish molecules and when alphabet includes P and S. (I ran the cell twice, because first call includes rpy2 initialization)

The results are almost identical, which is nice:

In [7]:
%time search_mz_candidates_rdisop(m, ['H', 'K', 'Na'], ppm_limit=1)[:10]

CPU times: user 252 ms, sys: 8 ms, total: 260 ms
Wall time: 257 ms


,em,error,mf,ppm,unsat,adduct
0,812.145770,-0.000009,C52H16N10O2,-0.011180,50,Na
1,796.171832,-0.000010,C46H37O7PS2,-0.012026,29,K
2,796.171866,0.000024,C31H41N8O7PS4,0.030120,16,K
3,834.127751,0.000027,C47H19N10O5P,0.032713,44,H
4,812.145741,-0.000038,C25H57N2O7P5S5,-0.046803,1,Na
5,796.171802,-0.000039,C30H47N4O9P3S3,-0.049307,11,K
6,796.171892,0.000050,C32H36N10O7P4,0.062813,22,K
7,812.145834,0.000055,C42H41N2O3PS5,0.067610,24,Na
8,834.127662,-0.000062,C45H30N4O7S3,-0.074305,33,H
9,834.127658,-0.000066,C30H35N10O9P5,-0.078684,21,H


I also tried a recent tool PFG (http://www.sciencedirect.com/science/article/pii/S0169743916300387), which can use multithreading. It computes somewhat different unsaturation values for formulas containing S or P, but that's fine: see http://fiehnlab.ucdavis.edu/projects/Seven_Golden_Rules/Ring-Double-Bonds

In [9]:
import os
import subprocess
pfg_executable_dir = os.path.expanduser("~/github/PFG")

def search_mz_candidates_pfg(mass, adducts, ppm_limit=5, charge=1):
    os.chdir(pfg_executable_dir)

    dfs = []
    for adduct in adducts:
        mass_ = mass - IsotopePattern(adduct).charged(charge).masses[0]
        cmd_line = ("OMP_NUM_THREADS=2 ./PFG -m {} -t {} " +\
                    "--C 0-100 --H 0-100 --N 0-10 --O 0-10 --S 0-5 --P 0-5 -r 'lewis'").format(mass_, ppm_limit)
        output = subprocess.check_output(cmd_line, shell=True)
        #print cmd_line, output
        results = open(os.path.join(pfg_executable_dir, "result.txt")).readlines()[1:]
        results = zip(*[s.split() for s in results])
        if not results:
            continue
        formula, _, mz, error, dbe = results
        
        #print result
        df = pd.DataFrame.from_dict({'mf': np.array(formula, dtype=str),
                                    'em': np.array(mz, dtype=float),
                                    'unsat': np.array(dbe, dtype=float).astype(int)
                                    })
   
        df['error'] = df['em'] - mass_
        df['ppm'] = df['error'] / mass_ * 1e6

        df['adduct'] = adduct
        dfs.append(df)
        
    return _combine_results(dfs)

With 2 threads, the speed is similar to that of ChemCalc (using more threads doesn't help significantly). I had to specify explicitly usage of 'lewis' golden rule on the command line to get the same results. 

In [10]:
%%time 
search_mz_candidates_pfg(m, ['H', 'K', 'Na'], ppm_limit=1)[:10]

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 61.3 ms


,em,error,mf,ppm,unsat,adduct
0,812.145770,-0.000009,C52H16N10O2,-0.010983,50,Na
1,796.171831,-0.000011,C46H37O7P1S2,-0.013414,34,K
2,796.171865,0.000023,C31H41N8O7P1S4,0.029290,25,K
3,834.127750,0.000026,C47H19N10O5P1,0.031712,45,H
4,812.145738,-0.000041,C25H57N2O7P5S5,-0.050385,16,Na
5,796.171801,-0.000041,C30H47N4O9P3S3,-0.051094,20,K
6,796.171890,0.000048,C32H36N10O7P4,0.060691,26,K
7,812.145833,0.000054,C42H41N2O3P1S5,0.066589,35,Na
8,834.127661,-0.000063,C45H30N4O7S3,-0.074986,39,H
9,834.127655,-0.000069,C30H35N10O9P5,-0.082179,26,H
